# `Grid` example

In this example we load a HDF5 (test) grid file into a corresponding `Grid` object, and use the inbuilt functionality to explore the grid.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps as cm
from matplotlib.colors import Normalize
from unyt import angstrom

from synthesizer.grid import Grid

This object takes the location of the grids on your system (`grid_dir`) and the name of the grid you wish to load `grid_name`). Here we load a simple test grid provided with the module (hence the relative path).

In [ ]:
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"

grid = Grid(grid_name, grid_dir=grid_dir)
print(grid)

A `Grid` can also take various arguments to limit the size of the grid, e.g. by isolating the `Grid` to a wavelength region of interest. This is particularly useful when making a large number of spectra from a high resolution `Grid`, where the memory footprint can become large.

### Passing a wavelength array

If you only care about a grid of specific wavelength values, you can pass this array and the `Grid` will automatically be interpolated onto the new wavelength array using [spectres](https://github.com/ACCarnall/SpectRes):

In [ ]:
# Define a new set of wavelengths
new_lams = np.logspace(2, 5, 1000) * angstrom

# Create a new grid
grid = Grid(grid_name, grid_dir=grid_dir, new_lam=new_lams)
print(grid.shape)

### Passing wavelength limits

If you don't want to modify the underlying grid resolution, but only care about a specific wavelength range, you can pass limits to truncate the grid at:

In [ ]:
# Create a new grid
grid = Grid(
    grid_name, grid_dir=grid_dir, lam_lims=(10**3 * angstrom, 10**4 * angstrom)
)
print(grid.shape)

## Plot a single grid point

We can plot the spectra at the location of a single point in our grid. First, we choose some age and metallicity

In [ ]:
# Return to the unmodified grid
grid = Grid(grid_name, grid_dir=grid_dir)

log10age = 6.0  # log10(age/yr)
Z = 0.01  # metallicity

We then get the index location of that grid point for this age and metallicity

In [ ]:
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=Z)

We can then loop over the available spectra (contained in `grid.spec_names`) and plot

In [ ]:
for spectra_type in grid.available_spectra:
    # Get `Sed` object
    sed = grid.get_sed_at_grid_point(grid_point, spectra_type=spectra_type)

    # Mask zero valued elements
    mask = sed.lnu > 0
    plt.plot(
        np.log10(sed.lam[mask]),
        np.log10(sed.lnu[mask]),
        lw=1,
        alpha=0.8,
        label=spectra_type,
    )

plt.legend(fontsize=8, labelspacing=0.0)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")

## Plot ionising luminosities

We can also plot properties over the entire age and metallicity grid, such as the ionising luminosity. 

In the examples below we plot ionising luminosities for HI and HeII

In [ ]:
fig, ax = grid.plot_specific_ionising_lum(ion="HI")

In [ ]:
fig, ax = grid.plot_specific_ionising_lum(ion="HeII")

## Resampling Grids

If you want to resample a grid after instantiation, you can apply the `intrep_spectra` method:

In [ ]:
# Define a new set of wavelengths
new_lams = np.logspace(2, 5, 10000) * angstrom

print("The old grid had dimensions:", grid.spectra["incident"].shape)

# Get the grid interpolated onto the new wavelength array
grid.interp_spectra(new_lam=new_lams)

print("The interpolated grid has dimensions:", grid.spectra["incident"].shape)

Note that this will overwrite the spectra and wavelengths read from the file *in place*.
To get back to the original arrays, a separate `Grid` can be instatiated without the modified wavelength array.

## Collapsing Grids

While most of the models within synthesizer are capable of handling higher dimensionality grids (i.e. grids with more dimensions than `age` and `metallicity`), other workflows might require some method to reduce the dimensionality. 

This functionality is provided via the `collapse()` method, which collapses the grid over a specified axis.
There are three ways to actually collapse the grid, specified by the `method` keyword argument:

- `marginalize` over the entire axis. This is useful if you don't know anything about this parameter, and just want to adopt the average over it. You can specify the function used to marginalize with the keyword argument `marginalize_function`; the default is `np.average`. 
- Pick the value `nearest` to a specified value. If you know the value of the parameter you want to use, you can collapse the grid by picking the value closest to your specified value. For this, you need to specify the `value` keyword argument. 
- `interpolate` to a specified value. Similar to `nearest`, but with a linear interpolation to your specified `value`. This is useful in workflows where you can't adopt a discrete value, but be warned that interpolating over a coarse grid can give unrealistic results. You can apply a transformation to the axis before interpolating, e.g. to interpolate in log-space rather than linear space, with the keyword argument `pre_interp_function`. 

For example, here we collapse the grid over the metallicity axis:

In [ ]:
print("The old grid had dimensions:", grid.spectra["incident"].shape)
print("and axes:", ", ".join(grid.axes))

# Collapse the grid to a single metallicity value
grid.collapse("metallicities", value=0.03, method="nearest")

print("The collapsed grid has dimensions:", grid.spectra["incident"].shape)
print("and axes:", ", ".join(grid.axes))

Note that `collapse()` will overwrite the `Grid` _in place_. You can restore the grid to its original dimensionality by re-loading from the HDF5 file:

In [ ]:
# Re-load the original grid
grid = Grid(grid_name, grid_dir=grid_dir)

## Converting a `Grid` into an `Sed`

Any of the spectra arrays stored within a `Grid` can be returned as `Sed` objects (see the `Sed` [docs](../sed/sed.rst)). This enables all of the analysis methods provide on an Sed to be used on the whole spectra grid. To do this we simply call `get_sed` with the spectra type we want to extract, and then use any of the included methods.

In [ ]:
# Get the sed object
sed = grid.get_sed(spectra_type="incident")

# Measure the balmer break for all spectra in the grid (limiting the output)
sed.measure_balmer_break()[5:10, 5:10]

## Working with flattened grids

Sometimes it's useful to work with flattened (i.e. one dimensional) versions of a grid of spectra, photometry etc. To facilitate this the `get_flattened_axes_values` method on `grid` can be used to get the flattend axes values.

In [ ]:
cm["plasma"](0.5)

In [ ]:
grid._axes_units["ages"]
grid._axes_values["ages"]

In [ ]:
# Get the balmer breaks of the entire grid
balmer_breaks = sed.measure_balmer_break()

# Get a flattened version of the Balmer break grid
flattened_balmer_breaks = balmer_breaks.flatten()

# Get the flattened version of the axes
flattend_axes_values = grid.get_flattened_axes_values()

# Normlise metallicities and create an array of colors
norm = Normalize(vmin=-4, vmax=-1.5, clip=True)
colors = cm["plasma"](norm(np.log10(flattend_axes_values["metallicities"])))

# Plot
plt.scatter(
    flattend_axes_values["ages"].to("Myr").value,
    flattened_balmer_breaks,
    c=colors,
)
plt.xscale("log")
plt.xlabel("ages/Myr")
plt.ylabel("Blamer break")
plt.show()